In [1]:
import os
os.environ["JAX_ENABLE_X64"] = "true"
import sys
sys.path.append('..')
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
from ipywidgets import interact
from ml_collections import ConfigDict
from models.CGLE import *
from filters import resamplers
from filters.filter import ParticleFilter
from filters.filter import ParticleFilterAll
from jax import config
jax.config.update("jax_enable_x64", True)
import numpy as np

float64


Initialisation of a twin experiment. 

In [2]:
signal_params = ConfigDict(CGLE_params)
ensemble_params = ConfigDict(CGLE_params)
signal_params.update(E=1,S=3,noise_magnitude = 0.01,stochastic_advection_basis='sin_sin')
ensemble_params.update(E=32,S=3,noise_magnitude = 0.01,stochastic_advection_basis='sin_sin')
key = jax.random.PRNGKey(0)

Now we specify the models, by calling the class.

In [3]:
signal_model = CGLE_SETD_KT_CM_JAX(signal_params)
ensemble_model = CGLE_SETD_KT_CM_JAX(ensemble_params)

initial_signal = initial_condition(signal_model.xx,signal_model.yy,signal_params.E,signal_params.initial_condition)#initial_condition(signal_model.x, signal_params.E, signal_params.initial_condition)
initial_ensemble = initial_condition(ensemble_model.xx, ensemble_model.yy, ensemble_params.E, ensemble_params.initial_condition)

print(ensemble_model.params,ensemble_model.params.nt)
available_resamplers = ", ".join(resamplers.keys())
print(available_resamplers)

E: 32
Forcing_basis_name: sin_sin
L: 100.0
Nt: 640
S: 3
alpha: 0.0
beta: 1.5
dt: 0.1
dx: 0.390625
equation_name: Complex Ginzburg-Landau
initial_condition: chebfun
method: Dealiased_SETDRK4_forced
noise_magnitude: 0.01
nt: 640
nx: 256
stochastic_advection_basis: sin_sin
tmax: 64.0
xmax: 50.0
xmin: -50.0
 640
multinomial, systematic, no_resampling, none, default


In [11]:
observation_spatial_frequency = 32
observation_locations = np.arange(signal_model.params.xmin,signal_model.params.xmax,observation_spatial_frequency,dtype=int)
observation_noise = 0.1
number_of_observations_time = 32 #ensemble_model.params.nt
observation_temporal_frequency = int(ensemble_model.params.nt/number_of_observations_time)
print(f"Observation temporal frequency: {observation_temporal_frequency}")
print(f"Observation spatial locations:{observation_locations}")
pf_systematic = ParticleFilterAll(
    n_particles = ensemble_params.E,
    n_steps = observation_temporal_frequency,
    n_dim = initial_signal.shape[-1],
    forward_model = ensemble_model,
    signal_model = signal_model,
    sigma = observation_noise,
    resampling="systematic",#'default',
    observation_locations = observation_locations,
)


Observation temporal frequency: 20
Observation spatial locations:[-50 -18  14  46]


To run the filter with all outputs, the input arrays need to have 3 dimensions.
They are (time, n_particles, space).

In [12]:
da_steps = number_of_observations_time
#print(initial_ensemble.shape, initial_signal.shape, da_steps)
initial_ensemble = initial_ensemble[None, ...] # adding the time dimension
initial_signal = initial_signal[None, ...] # adding the time dimension
key = jax.random.PRNGKey(1) # random key for JAX
final, all = pf_systematic.run(initial_ensemble, initial_signal, da_steps ,key) #the final input is scan length? 


(1, 1, 1, 1, 256, 256)
(3, 256, 256) (1, 3) (1, 3)
(1, 1, 1, 32, 256, 256)
(3, 256, 256) (32, 3) (32, 3)
(np.int64(-50), np.int64(-18), np.int64(14), np.int64(46))


TypeError: sub got incompatible shapes for broadcasting: (2097152,), (65536,).

In [ ]:
print("final shapes:")
for i, arr in enumerate(final):
    print(f"final[{i}]:", arr.shape)

print("\nall shapes:")
for i, arr in enumerate(all):
    print(f"all[{i}]:", arr.shape)

The two outputs are tuples. The first tuple has length 2 and contains the final ensemble and the final signal arrays.
The second tuple has length three and contains all outputs through time of the ensemble, signal and observation.

Put in the initial condition.

In [ ]:
# particles
print(initial_ensemble.shape)
print(all[0].shape)
all_0_reshaped = jnp.reshape(all[0], (-1, all[0].shape[2], all[0].shape[3]))
print(all_0_reshaped.shape)

particles =jnp.concatenate([initial_ensemble[...], all_0_reshaped], axis=0)
print(particles.shape)

## signal 
print(all[1].shape)
all_1_reshaped = jnp.reshape(all[1], (-1, all[1].shape[2], all[1].shape[3]))
print(all_1_reshaped.shape)
print(initial_signal.shape)
signal = jnp.concatenate([initial_signal[...], all_1_reshaped], axis=0)
print(signal.shape)

## observations
print(all[2].shape)
# observations = jnp.concatenate([initial_signal[...], all[2]], axis=0)
observations = all[2][:,:, observation_locations]
print(observations.shape)


In [ ]:
def plot(time):
    plt.plot(signal_model.x, signal[time,0,:], color='k',label='signal')
    plt.plot(signal_model.x, particles[time,:,:].T, color='b',label='particles',linewidth=0.1)
    if (time ) % observation_temporal_frequency == 0 and time !=0:
        plt.plot(signal_model.x[observation_locations], observations[int(time/observation_temporal_frequency),0,:], 'ro',label='observations')
    plt.show()

interact(plot, time=(0, ensemble_model.params.nt - 1, 1))

In [ ]:
ensemble_params_none  = ConfigDict(KS_params_SALT)
ensemble_params_none.update(E=128,P=32,noise_magnitude = 0.01,stochastic_advection_basis='sin')
ensemble_model_none   = ETD_KT_CM_JAX_Vectorised(ensemble_params_none)
initial_signal_none   = initial_condition(signal_model.x, signal_params.E, signal_params.initial_condition)
initial_ensemble_none = initial_condition(ensemble_model.x, ensemble_params.E, ensemble_params.initial_condition)

pf_none = ParticleFilterAll(
    n_particles = ensemble_params.E,
    n_steps = observation_temporal_frequency,
    n_dim = initial_signal.shape[-1],
    forward_model = ensemble_model,
    signal_model = signal_model,
    sigma = observation_noise,
    resampling="no_resampling",#'default',
    observation_locations = observation_locations,
)
final_none, all_none = pf_none.run(initial_ensemble, initial_signal, da_steps,key) #the final input is scan length?
initial_ensemble_none = initial_ensemble_none[None, ...] # adding the time dimension
initial_signal_none = initial_signal_none[None, ...] # adding the time dimension
key = jax.random.PRNGKey(1) # random key for JAX

In [ ]:
final_none, all_none = pf_none.run(initial_ensemble_none, initial_signal_none, da_steps,key)

In [ ]:
all_none_0_reshaped = jnp.reshape(all_none[0], (-1, all_none[0].shape[2], all_none[0].shape[3]))
particles_none =jnp.concatenate([initial_ensemble_none[...], all_none_0_reshaped], axis=0)
print(all_none_0_reshaped.shape)

def plot_member_none(e):
    # Calculate observation times (y-axis) and locations (x-axis)
    obs_times = np.arange(0, particles_none.shape[0], observation_temporal_frequency )
    obs_x = signal_model.x[observation_locations]

    # For each observation time, plot scatter points at observation locations
    for t in obs_times:
        plt.scatter(observation_locations, [t]*len(observation_locations), color='red', s=10, label='Observations' if t == obs_times[0] else "")

    # Optional: only show one legend entry
    handles, labels = plt.gca().get_legend_handles_labels()
    if handles:
        plt.legend()
    plt.imshow(particles_none[:, e, :], aspect='auto', origin='lower')
    plt.title(f'Ensemble member {e} (no resampling)')
    plt.xlabel('Space')
    plt.ylabel('Time')
    plt.show()

interact(plot_member_none, e=(0, all_none_0_reshaped.shape[1] - 1))

In [ ]:
def plot(time):
    plt.plot(signal_model.x, signal[time,0,:], color='k',label='signal')
    plt.plot(signal_model.x, particles[time,:,:].T, color='b',label='particles',linewidth=0.1)
    plt.plot(signal_model.x, particles_none[time,:,:].T, color='g',label='particles (no resampling)',linewidth=0.1)
    if (time ) % observation_temporal_frequency == 0 and time !=0:
        plt.plot(signal_model.x[observation_locations], observations[int(time/observation_temporal_frequency),0,:], 'ro',label='observations')
    plt.show()

interact(plot, time=(0, ensemble_model.params.nt - 1, 1))

In [ ]:
from metrics.ensemble import rmse, crps_internal
subsample = 5# we sometimes have to subsample the data for speed issues can go as low as 5, for a 
signal_sub = signal[::subsample,:,:]
particles_sub = particles[::subsample,:,:]
particles_none_sub = particles_none[::subsample,:,:]
# Compute RMSE and CRPS for the ensemble (with resampling)
rmse_ensemble = rmse(signal_sub,particles_sub)
crps_ensemble = crps_internal(signal_sub, particles_sub)#crps(signal,particles)

# Compute RMSE and CRPS for the ensemble (no resampling)
rmse_none = rmse(signal_sub,particles_none_sub)
crps_none = crps_internal(signal_sub,particles_none_sub)
print("RMSE (systematic):", rmse_ensemble.average())
print("CRPS (systematic):", crps_ensemble)
print("RMSE (no resampling):", rmse_none)
print("CRPS (no resampling):", crps_none)


In [ ]:
dpi=300
plt.figure(figsize=(7, 4), dpi=dpi)

# Plot RMSE with markers only at observation frequency
obs_idx = np.arange(0, len(rmse_ensemble), observation_temporal_frequency//subsample ) #Observation temporal frequency: 125
# should result in 32 observations for 
#obs_idx = np.arange(0, len(rmse_ensemble), observation_temporal_frequency)
plt.plot(rmse_ensemble, color='black', marker='None', label='Particle Filter')
plt.plot(obs_idx, rmse_ensemble[obs_idx], color='black', marker='o', linestyle='None')

plt.plot(rmse_none, color='blue', marker='None', label='No Particle Filter')
plt.plot(obs_idx, rmse_none[obs_idx], color='blue', marker='s', linestyle='None')
# plt.plot(rmse_ensemble, color='black', marker='o', label='Particle Filter')
# plt.plot( rmse_none, color='blue', marker='s', label='No Particle Filter')
plt.xlabel('Lead Time (DA step)')
plt.ylabel('RMSE')
plt.title('RMSE')
plt.legend()
plt.savefig('/Users/jmw/Documents/GitHub/Particle_Filter/Saving/EX3a_KdV_PF_NPF_RMSE.png', bbox_inches='tight', dpi=dpi)

plt.show()

# Plot CRPS with markers only at observation frequency
plt.figure(figsize=(7, 4), dpi=dpi)
plt.plot(crps_ensemble, color='black', marker='None', label='Particle Filter')
plt.plot(obs_idx, crps_ensemble[obs_idx], color='black', marker='o', linestyle='None')

plt.plot(crps_none, color='blue', marker='None', label='No Particle Filter')
plt.plot(obs_idx, crps_none[obs_idx], color='blue', marker='s', linestyle='None')

plt.xlabel('Lead Time (DA step)')
plt.ylabel('CRPS')
plt.title('CRPS')
plt.legend()
plt.savefig('/Users/jmw/Documents/GitHub/Particle_Filter/Saving/EX3a_KdV_PF_NPF_CRPS.png', bbox_inches='tight', dpi=dpi)

plt.show()